In [20]:
import json
import glob
import pandas as pd

# Find all results.json files
results_files = glob.glob("../logs/binary/*/version_*/results.json")

In [36]:
# List to hold all rows
rows = []

# Process each file
for file_path in results_files:
    version = file_path.split("/")[-2].split("_")[-1]  # Extract 'version_X'
    fusion = file_path.split("/")[-3]  # Extract 'fusion'
    with open(file_path, "r") as f:
        data = json.load(f)

    # Flatten the structure: (version, split, metric, value)
    for split in ["train", "val", "test"]:
        if split in data:
            row = {"fusion": fusion, "version": version, "split": split}
            row.update(data[split])
            rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows)

# Split DataFrame
train = df[df["split"] == "train"].drop(columns=["split"])
dev = df[df["split"] == "val"].drop(columns=["split"])
test = df[df["split"] == "test"].drop(columns=["split"])

# Sorting: Best EER first
train = train.sort_values(by="auc", ascending=False).reset_index(drop=True)
dev = dev.sort_values(by="auc", ascending=False).reset_index(drop=True)
test = test.sort_values(by="auc", ascending=False).reset_index(drop=True)

In [37]:
maximize_metrics = ["acc", "ap", "prec", "rec", "f1", "auc"]
minimize_metrics = ["loss"]

In [38]:
def highlight_best_per_column(s):
    if s.name in maximize_metrics:
        is_best = s == s.max()
    elif s.name in minimize_metrics:
        is_best = s == s.min()
    else:
        is_best = [False] * len(s)
    return [
        (
            "background-color: red"
            if v and (s.name == "auc" or s.name == "ap")
            else "background-color: green" if v else ""
        )
        for v in is_best
    ]

In [39]:
# Display with correct highlighting
print("Train results:")
display(train.style.apply(highlight_best_per_column, axis=0))

print("Dev results:")
display(dev.style.apply(highlight_best_per_column, axis=0))

print("Test results:")
display(test.style.apply(highlight_best_per_column, axis=0))

Train results:


,fusion,version,acc,prec,rec,f1,ap,auc
0,CAFF,0,0.999789,0.999581,1.000000,0.999790,1.000000,1.000000
1,CFF,0,0.999789,0.999581,1.000000,0.999790,1.000000,1.000000
2,GFF,0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,AFF,0,0.999683,0.999372,1.000000,0.999686,1.000000,1.000000
4,IAFF,0,0.999789,0.999581,1.000000,0.999790,1.000000,1.000000
5,MMD,0,0.999683,0.999581,0.999790,0.999686,1.000000,1.000000


Dev results:


,fusion,version,acc,prec,rec,f1,ap,auc
0,MMD,0,0.988539,0.986842,0.990566,0.988701,0.999703,0.999693
1,CAFF,0,0.987584,0.981378,0.994340,0.987816,0.999602,0.999591
2,IAFF,0,0.990449,0.983271,0.998113,0.990637,0.999581,0.999569
3,CFF,0,0.988539,0.981413,0.996226,0.988764,0.999466,0.999453
4,GFF,0,0.989494,0.979667,1.000000,0.989729,0.999446,0.999438
5,AFF,0,0.988539,0.981413,0.996226,0.988764,0.999414,0.999409


Test results:


,fusion,version,acc,prec,rec,f1,ap,auc
0,CAFF,0,0.645483,0.601662,0.919172,0.727273,0.800989,0.781714
1,IAFF,0,0.666094,0.622836,0.889112,0.732526,0.796637,0.773692
2,GFF,0,0.649261,0.608875,0.889112,0.722780,0.797891,0.772115
3,AFF,0,0.639643,0.600901,0.891116,0.717783,0.795333,0.768440
4,CFF,0,0.657162,0.613564,0.900468,0.729832,0.781948,0.762229
5,MMD,0,0.641704,0.603370,0.885104,0.717574,0.763044,0.756379
